In [ ]:
import header
paths = header.setup_environment()
for k,v in paths.items():
    print(k, '->', v)

import os
import pandas as pd

def PaDiff1m(T=0):
    PaPerM = 12.67 - 0.041*T
    return PaPerM

def correctForHeightAboveWaterFt(heightFt=0, T=0):
    # correct down to height 0m
    heightM = heightFt * 0.3048
    psi_shift = (PaDiff1m(T=T) * heightM) * pa2psi
    return psi_shift

def correctForDepthBelowWaterFt(depthFt=0, T=0):
    # correct up to 0m
    psi_shift = abs(depthFt)/2.31
    return psi_shift

INPUTDIR = os.path.join(paths['new_data'], '03_merge_inventories')
dfall = pd.read_csv(os.path.join(INPUTDIR, 'all2.csv'))
std_temperature = 20.0
std_passcals = 101325
pa2psi = 0.000145038
std_psi = std_passcals * pa2psi


import matplotlib.pyplot as plt


In [ ]:

for subdir in ['Baro', '20hz', '100hz']:
    dfsub = dfall.copy()[dfall['subdir']==subdir]
    dfsub['datetime'] = pd.to_datetime(dfsub['TIMESTAMP'])
    dfsub.dropna(how='all', axis=1, inplace=True) 
    dfsub.drop(dfsub.columns[dfsub.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    print(subdir, dfsub.columns)
    display(dfsub)
    if 'AirPressureShallow' in dfsub.columns:
        dfsub['AirPressureShallow']=dfsub['AirPressureShallow']-std_psi
        dfsub['AirPressureDeep']=dfsub['AirPressureDeep']-std_psi
        dfsub['AirPressureDiff']=dfsub['AirPressureDeep']-dfsub['AirPressureShallow']
        dfsub['AirPressureDiffPa'] = dfsub['AirPressureDiff']/pa2psi

        dfsub.plot(x='datetime', y=['AirPressureShallow', 'AirPressureDeep', 'AirPressureDiff'], kind='line', ylim=[-.3, .3], ylabel='PSI relative to STP')
        dfsub.plot(x='datetime', y=['AirPressureDiff'], kind='line', ylabel='PSI difference deep - shallow', style='.')
        dfsub.plot(x='datetime', y=['AirPressureDiffPa'], kind='line', ylabel='Pa difference deep - shallow', style='.')
    if 'Therm(6)' in dfsub.columns:
        dfsub2 = dfsub.copy()
        dfsub2.dropna(subset=['Therm(6)'], inplace=True)
        dfsub2.plot(x='datetime', y=['Therm(1)', 'Therm(2)', 'Therm(3)', 'Therm(4)', 'Therm(5)', 'Therm(6)'], kind='line', ylabel='Temperature (C)').legend(bbox_to_anchor=(1.0,1.0), fontsize='small')


In [ ]:
import header
paths = header.setup_environment()
import os
#import sys
#import glob
#import numpy as np
import pandas as pd
#from obspy.core import read, Stream, UTCDateTime
#import FDSNtools
#import wrappers
#import SDS
import libWellData as LLE

# Parse lookuptable
transducersDF = LLE.get_transducers_dataframe(paths)
#lookuptableDF = LLE.removed_unnamed_columns(pd.read_csv(paths['lookuptable']))
#lookuptableDF.to_csv('lookuptable_backup.csv')
#lookuptableDF = lookuptableDF.sort_values(by=['starttime'])
#lookuptableDF['miniseed'] = False
print(paths)
print('- Applying calibration equations')
dfcorrected=dfall.copy()
for col in dfcorrected.columns:
    #print(col)
    if isinstance(col,str) and (col[0:2]=='12' or col[0:2]=='21'):
        this_transducer = transducersDF[(transducersDF['serial']) == col]
        #print(this_transducer)
        if len(this_transducer.index)==1:
            this_transducer = this_transducer.iloc[0].to_dict()
            #print(this_transducer)
            dfcorrected[col] = LLE.compute_psi(dfcorrected[col].to_numpy(), this_transducer)

dfbaro = dfcorrected.copy()[dfcorrected['subdir']=="Baro"]
dfbaro['datetime'] = pd.to_datetime(dfbaro['TIMESTAMP'])
dfbaro.dropna(how='all', axis=1, inplace=True) 
dfbaro.drop(dfbaro.columns[dfbaro.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

df100hz = dfcorrected.copy()[dfcorrected['subdir']=="100hz"]
df100hz['datetime'] = pd.to_datetime(df100hz['TIMESTAMP'])
df100hz.dropna(how='all', axis=1, inplace=True) 
df100hz.drop(df100hz.columns[df100hz.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

ax1 = dfbaro.plot(x='datetime', y=['AirPressureShallow', 'AirPressureDeep'], kind='line')
df100hz.plot(ax=ax1, x='datetime', y=['1226420', '1226429'], kind='line', ylim=[std_psi-0.5, std_psi+0.3], style='.-')
# psi[i] = ((dig[i]-d['dig0']) * d['gf'] + (d['tt']-d['tt0'])*d['tf']+(d['bp0']-d['bp']))
"""
        # Intermediate well (HOF-IW00061)
        transducer6 = {'serial':'AirPressureDeep', 'Fs':100, 'sensor':'barometer','shielding':'none',
               'range_kPa_low':100,'range_kPa_high':100,'media':'air', 'type':'pressure', 
               'model':'Keller 0507.01401.051311.07','set_depth_ft':4.46, 'id':'6I.0XXXX.XX.HDH'
              }
        transducers.append(transducer6)
        transducer7 = {'serial':'1226429', 'Fs':100, 'sensor':'vibrating_wire','shielding':'none',
               'range_kPa_low':70,'range_kPa_high':170,'media':'water', 'type':'level', 
               'model':'Geokon 4500AL','set_depth_ft':3.71,
               'dig0':9800, 'gf':-0.006428, 'tt':22.6, 'tt0':21.6, 'tf':-0.002384, 
               'bp':0.0, 'bp0':14.298, 'id':'6I.12264.29.HDD'          
              }

        # Shallow well (HOF-IW0006S)
        transducer1 = {'serial':'AirPressureShallow', 'Fs':100, 'sensor':'barometer','shielding':'none',
               'range_kPa_low':100,'range_kPa_high':100,'media':'air', 'type':'pressure', 
               'model':'Keller 0507.01401.051311.07','set_depth_ft':4.46, 'id':'6S.02374.88.HDH'
              } # serial 237488
        transducers.append(transducer1)
        transducer2 = {'serial':'1226420', 'Fs':100, 'sensor':'vibrating_wire','shielding':'none',
               'range_kPa_low':70,'range_kPa_high':170,'media':'water', 'type':'level', 
               'model':'Geokon 4500AL','set_depth_ft':3.81,
               'dig0':9751, 'gf':-0.006458, 'tt':21.6, 'tt0':21.3, 'tf':-0.008795, 
               'bp':0.0, 'bp0':14.298, 'id':'6S.12264.20.HDD'
              }
"""
df100hz['1226420shifted']=df100hz['1226420']+(30-21.3)*-0.008795+correctForHeightAboveWaterFt(heightFt=3.81, T=30)
df100hz['1226429shifted']=df100hz['1226429']+(30-21.6)*-0.002384+correctForHeightAboveWaterFt(heightFt=3.71, T=30)
ax2 = dfbaro.plot(x='datetime', y=['AirPressureShallow', 'AirPressureDeep'], kind='line')
df100hz.plot(ax=ax2, x='datetime', y=['1226420shifted', '1226429shifted'], kind='line', ylim=[std_psi-0.5, std_psi+0.3], style='.-')


In [ ]:
print(correctForHeightAboveWaterFt(heightFt=3.81, T=29))
print(correctForHeightAboveWaterFt(heightFt=3.81, T=32))
print((29-21.3)*-0.008795-(32-21.3)*-0.008795)

print(correctForHeightAboveWaterFt(heightFt=3.71, T=29))
print(correctForHeightAboveWaterFt(heightFt=3.71, T=32))
print((29-21.3)*-0.002384)
print((32-21.3)*-0.002384)

display(transducersDF)

In [ ]:
# now let's pick a 24-hour period, and find all files from the all2.csv summary file within that time window, and plot them
dfday = dfall.copy()
dfday.dropna(how='all', axis=1, inplace=True) 
dfday.drop(dfday.columns[dfday.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
dfday['datetime'] = pd.to_datetime(dfday['TIMESTAMP'])
dfday = dfday.query('datetime > 202211010759 & datetime < 202211011159')
print(dfday.columns)
display(dfday['basename'])

# load and plot all these files
dfdata = {}

for index, row in dfday.iterrows():
    subdir = row['subdir']
    fullpath = os.path.join(paths['new_data'], '03_merge_inventories', str(row['uploaddate']), row['subdir'], row['basename'])
    if not subdir in dfdata:
        dfdata[subdir] = pd.DataFrame()
    print(fullpath)
    if os.path.isfile(fullpath):
        print('- loading')
        thisdf = pd.read_csv(fullpath)
        thisdf.dropna(how='all', axis=1, inplace=True) 
        thisdf.drop(thisdf.columns[thisdf.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
        thisdf['datetime'] = pd.to_datetime(thisdf['TIMESTAMP'])
        dfdata[subdir] = pd.concat([dfdata[subdir], thisdf])
    else:
        print('- not found')

for k in dfdata.keys():
    thisdf = dfdata[k].copy()
    thisdf.set_index('datetime', inplace=True)
    thisdf.drop(['TIMESTAMP', 'RECORD'], axis=1, inplace=True)
    thisdf.plot()

print(dfdata)

In [ ]:
numSamplesToPlot=100
for k in dfdata.keys():
    thisdf = dfdata[k]
    samples_per_window=int(len(thisdf)/numSamplesToPlot)
    if not k=='Baro':
        continue
    for col in ['AirPressureShallow', 'AirPressureDeep']:
        if col in thisdf.columns and 'Air' in col or '12' in col or '21' in col:
            thisdf['moving_' + col] = thisdf[col].rolling(window=samples_per_window).median()
    try:
        thisdf.plot(x='datetime', y=['moving_AirPressureShallow', 'moving_AirPressureDeep'], kind='line')
        plt.show()
    except:
        pass

In [ ]:
for k in dfdata.keys():
    thisdf = dfdata[k].copy()
    #thisdf.set_index('datetime', inplace=True)
    print(thisdf.columns)
    thisdf.drop(['TIMESTAMP', 'RECORD'], axis=1, inplace=True)
    resampleddf = thisdf.resample('10s', on='datetime').median()
    for col in resampleddf.columns:
        resampleddf[col] = resampleddf[col]-resampleddf[col][0]
    resampleddf.plot(kind='line')
    plt.show()

In [ ]:
# RSAM simulation
import obspy
st = obspy.Stream()
import numpy as np
sample_interval = 1
for k in dfdata.keys():
    thisdf = dfdata[k].copy()
    thisdf.set_index('datetime', inplace=True)
    thisdf.drop(['TIMESTAMP', 'RECORD'], axis=1, inplace=True)
    for col in thisdf.columns:
        tr = obspy.Trace(data=thisdf[col].to_numpy())
        nan_positions = np.where(np.isnan(tr.data))
        tr.data = np.nan_to_num(tr.data, nan=0.0)
        tr.detrend('linear')
        tr.filter('highpass', freq=0.05)
        tr.data[nan_positions] = np.nan
        thisdf[col] = abs(tr.data)
    print(thisdf.columns)
    thisdf.reset_index(inplace=True)
    #resampleddf = thisdf.resample('10s', on='datetime').median()
    resampleddf = thisdf.resample(f'{sample_interval}s', on='datetime').median()
    print(resampleddf)
    #for col in resampleddf.columns:
    #    resampleddf[col] = resampleddf[col]-resampleddf[col][0]
    resampleddf.plot(kind='line')
    plt.show()
    for col in resampleddf:
        tr2 = obspy.Trace(data=resampleddf[col].to_numpy())
        tr2.data = np.nan_to_num(tr2.data, nan=0.0)
        tr2.id = f'{col[0:2]}.{col[2:7]}.{col[7:9]}.{col[9:]}'
        tr2.stats.delta = float(sample_interval)
        tr2.stats.starttime=obspy.UTCDateTime(resampleddf.index[0])
        st.append(tr2)

st.plot(equal_scale=False);
print(st)

In [ ]:
#st.filter('bandpass', freqmin=10, freqmax=20)  # optional prefiltering
print(st)
from obspy.signal.trigger import coincidence_trigger

st2 = st.copy()

trigs = coincidence_trigger("recstalta", 2, 1, st2, int(len(st2)/2), sta=sample_interval*5, lta=sample_interval*100)
for thistrig in trigs:
    display(thistrig)
    noisewindow = st2.copy().trim(starttime=thistrig['time']-60, endtime=thistrig['time']-10)
    for tr in noisewindow:
        if not np.all(tr.data):
            thistrig['trace_ids'].remove(tr.id)
    if len(thistrig['trace_ids'])>len(st2)/2:
        st3 = st2.copy().trim(starttime=thistrig['time']-60, endtime=thistrig['time']+60+thistrig['duration'])
        st3.plot(equal_scale=False);